In [1]:
from elasticsearch_dsl import analyzer, connections, Index, Document, Text, Search
from elasticsearch.helpers import bulk
from elasticsearch import Elasticsearch

In [2]:
connections.create_connection(hosts="http://192.168.0.28:9200", verify_certs=False)  # Running on separate machine on local-network due to RAM constraints
es = Elasticsearch(["http://192.168.0.28:9200"])

## Zadanie 1

*Use the FIQA-PL dataset that was used in lab 1 and lab lab 2 (so we need the passages, the questions and their relations).*

In [3]:
import datasets as ds
import pandas as pd

/Users/kkafara/studies/9_term/pjn/repo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from IPython.display import clear_output
ds_name = "clarin-knext/fiqa-pl"
res_ds_name = "clarin-knext/fiqa-pl-qrels"

queries_df = ds.load_dataset(ds_name, "queries")["queries"].to_pandas()
corpus_df = ds.load_dataset(ds_name, "corpus")["corpus"].to_pandas()
response_df = ds.load_dataset(res_ds_name)

response_train_df = response_df['train'].to_pandas()
response_validation_df = response_df['validation'].to_pandas()
response_test_df = response_df['test'].to_pandas()

clear_output()

In [5]:
queries_df.head(3)

,_id,title,text
0,0,,Co jest uważane za wydatek służbowy w podróży ...
1,4,,Wydatki służbowe - ubezpieczenie samochodu pod...
2,5,,Rozpoczęcie nowego biznesu online


In [6]:
queries_df._id = queries_df._id.astype(int)
corpus_df._id = corpus_df._id.astype(int)
# response_train_df._id = response_train_df._id.astype(int)
# response_validation_df._id = response_validation_df._id.astype(int)
# response_test_df._id = response_test_df._id.astype(int)

In [7]:
corpus_df.head(3)

,_id,title,text
0,3,,"Nie mówię, że nie podoba mi się też pomysł szk..."
1,31,,Tak więc nic nie zapobiega fałszywym ocenom po...
2,56,,Nigdy nie możesz korzystać z FSA dla indywidua...


In [8]:
response_train_df.head(3)

,query-id,corpus-id,score
0,0,18850,1
1,4,196463,1
2,5,69306,1


## Zadania 2, 3

Create a dataset of positive and negative sentence pairs. In each pair the first element is a question and the second element is a passage. Use the relations to mark the positive pairs (i.e. pairs where the question is answered by the passage). Use your own strategy to mark negative pairs (i.e. you can draw the negative examples, but there are better strategies to define the negative examples). The number of negative examples should be much larger than the number of positive examples.


The dataset from point 2 should be split into training, evaluation and testing subsets.


Stworzenie pozytywnych par jest proste - korzystamy z danych które mamy w `reponse_df`.

In [9]:
def create_positive_pairs_from_df(df: pd.DataFrame) -> pd.DataFrame:
    questions = []
    answers = []
    for index, row in df.iterrows():
        question = queries_df.loc[queries_df._id == row['query-id']].iloc[0]['text']
        positive_answer = corpus_df.loc[corpus_df._id == row['corpus-id']].iloc[0]['text']
        questions.append(question)
        answers.append(positive_answer)
    return pd.DataFrame({
        'query': questions,
        'answer': answers
    })

pairs_positive_train_df = create_positive_pairs_from_df(response_train_df)
pairs_positive_validation_df = create_positive_pairs_from_df(response_validation_df)
pairs_positive_test_df = create_positive_pairs_from_df(response_test_df)

In [10]:
pairs_positive_train_df.head(2)

,query,answer
0,Co jest uważane za wydatek służbowy w podróży ...,Wytyczne IRS dotyczące tematu. Ogólnie rzecz b...
1,Wydatki służbowe - ubezpieczenie samochodu pod...,Co do zasady musisz wybrać pomiędzy odliczenie...


In [11]:
pairs_positive_train_df.shape, pairs_positive_test_df.shape, pairs_positive_validation_df.shape

((14166, 2), (1706, 2), (1238, 2))

Idąc za sugestią z zajęć, pary negatywne wygeneruję jako "złe odpowiedzi" zwrócone przez ES'a -- albo ustawię próg na score, albo 
będę pytał o np. 20 wyników i wezmę 5 najgorszych. Jeszcze zobaczę jak to wyjdzie w praniu.

Najpierw wgrajmy dane do ES'a

In [12]:
basic_analyzer = analyzer('basic_analyzer', tokenizer='standard', filter=['lowercase', 'morfologik_stem', 'lowercase'])

In [13]:
class Article(Document):
    body = Text(analyzer=basic_analyzer)

    class Index:
        name = "corpus"

Article.init()

In [14]:
# articles = []
# for _, row in corpus_df.iterrows():
#     article = Article(meta={'id': row._id}, body=row.text).to_dict(include_meta=True)
#     articles.append(article)

# bulk(es, articles)

In [15]:
from typing import Any, Dict

def body_for_query(query: str) -> Any:
    return {"query": {"match": {"body": {"query": query}}}}

Zobaczmy czy to działa na przykładowym query

In [16]:
query = pairs_positive_train_df.iloc[0]['query']
query

'Co jest uważane za wydatek służbowy w podróży służbowej?'

In [17]:
from pprint import pprint

# Dobrałem tą wartość dla przykładowego query tak, żeby faktycznie zwrócone teksty nie zawierały informacji które chociaż częściowo
# odpowiadają na pytanie. Ciągle jest to wybór mocno arbitralny i może zdarzyć się tak, że to nie do końca będą negatywy - a jedynie
# złej jakości pozytywy.
result_count_upper_limit = 1000

result = es.search(index='corpus', body=body_for_query(query), size=result_count_upper_limit)

In [18]:
list(map(lambda hit: hit['_score'], result.body['hits']['hits'][-5:]))

[6.8649077, 6.862724, 6.859345, 6.8590593, 6.858466]

Powiedzmy, że score poniżej 7 daje już satysfakcjonująco złe odpowiedzi

In [19]:
score_upper_bound = 7

Poniżej trochę kombinuję z batchowaniem requestów, bo ESa mam postawionego na osobnej maszynie z powodu "ramożerności" ESa,
a bez batchowania to potrafi się liczyć & przesyłać długo...

In [20]:
from tqdm import tqdm
from elasticsearch_dsl.query import Query
from elasticsearch_dsl import MultiSearch, Search

def take_from_generator(generator, n):
    while True:
        batch = []
        i = 0
        while i < n:
            try:
                elem = next(generator)
                batch.append(elem)
                i += 1
            except StopIteration:
                break
        if len(batch) > 0:
            yield batch
        else:
            break

def create_negative_pairs_from_df(df: pd.DataFrame) -> pd.DataFrame:
    all_query_texts = []
    neg_answers = []
    batch_size = 256
    with tqdm(total=df.shape[0] // batch_size) as pbar:
        for row_batch in take_from_generator(df.iterrows(), batch_size):
            # multiquery = MultiSearch(index='corpus', using=es)
            queries = (queries_df.loc[queries_df._id == row_tuple[1]['query-id']].iloc[0]['text'] for row_tuple in row_batch)
            es_queries = (body_for_query(query) for query in queries)
    
            query_texts = []
            searches = []
            for es_query in es_queries:
                searches.append({"index": "corpus"})
                searches.append(es_query)
                query_texts.append(es_query['query']['match']['body']['query'])
    
            responses = es.msearch(index='corpus', body=searches)
            
            for text, response in zip(query_texts, responses['responses']):
                last = [ans['_source']['body'] for ans in response['hits']['hits'] if ans['_score'] < score_upper_bound]
                if len(last) > 5:
                    last = last[-5:]
                elif len(last) == 0:
                    last = [ans['_source']['body'] for ans in response['hits']['hits'][-5:]]
    
                for ans in last:
                    all_query_texts.append(text)
                    neg_answers.append(ans)
            pbar.update(1)

    return pd.DataFrame({
        'query': all_query_texts,
        'answer': neg_answers
    })

In [21]:
pairs_negative_train_df = create_negative_pairs_from_df(response_train_df)

56it [00:33,  1.65it/s]                                                                                                                                                                                 


In [22]:
pairs_negative_validation_df = create_negative_pairs_from_df(response_validation_df)
pairs_negative_test_df = create_negative_pairs_from_df(response_test_df)

5it [00:03,  1.40it/s]                                                                                                                                                                                  
7it [00:04,  1.67it/s]                                                                                                                                                                                  


In [23]:
pairs_negative_train_df.head(10)

,query,answer
0,Co jest uważane za wydatek służbowy w podróży ...,"Witamy na Carrental.com, oferujemy naszym klie..."
1,Co jest uważane za wydatek służbowy w podróży ...,>I wciąż wystarczające zapotrzebowanie na zape...
2,Co jest uważane za wydatek służbowy w podróży ...,"„To, co robią, jest złe. IRS i państwo mogą ni..."
3,Co jest uważane za wydatek służbowy w podróży ...,„Typowe dla dużych firm. Moja firma nie ogłasz...
4,Co jest uważane za wydatek służbowy w podróży ...,"Nie mówisz, w jakim kraju mieszkasz. Jeśli są ..."
5,Wydatki służbowe - ubezpieczenie samochodu pod...,"Nie ma prawa, które wymaga posiadania oddzieln..."
6,Wydatki służbowe - ubezpieczenie samochodu pod...,„Zwroty wydatków służbowych na ogół nie podleg...
7,Wydatki służbowe - ubezpieczenie samochodu pod...,"W porządku, IRS Publikacja 463: Podróże, rozry..."
8,Wydatki służbowe - ubezpieczenie samochodu pod...,Zapoznałem się z przykładami w publikacji 463 ...
9,Wydatki służbowe - ubezpieczenie samochodu pod...,"Z wyjątkiem tego, że ubezpieczenie większości ..."


In [24]:
pairs_negative_train_df.tail(10)

,query,answer
70814,Czy strata kapitałowa w tradycyjnej IRA i Roth...,1) Dlaczego nie mógłbym wpłacać składek na kon...
70815,Czy strata kapitałowa w tradycyjnej IRA i Roth...,"Tak, możesz wpłacać niepodlegające odliczeniu ..."
70816,Czy strata kapitałowa w tradycyjnej IRA i Roth...,Rollover IRA to tradycyjna IRA. Twoje składki ...
70817,Czy strata kapitałowa w tradycyjnej IRA i Roth...,"„W dzisiejszych czasach, jak wspomniał JoeTaxp..."
70818,Czy strata kapitałowa w tradycyjnej IRA i Roth...,Musisz złożyć wniosek jako żonaty/zamężna za r...
70819,Sprzedaż akcji w celu zrekompensowania innych ...,"""Po raz kolejny udzielam mądrej rady - """"Nie p..."
70820,Sprzedaż akcji w celu zrekompensowania innych ...,Sprzedaż akcji tworzy zysk kapitałowy. Można t...
70821,Sprzedaż akcji w celu zrekompensowania innych ...,"Obawiam się, że nie dostaniesz tu żadnych dobr..."
70822,Sprzedaż akcji w celu zrekompensowania innych ...,Z paragrafu 1091 IRS. Strata z tytułu sprzedaż...
70823,Sprzedaż akcji w celu zrekompensowania innych ...,"Nie jesteś osobą ani podmiotem, przeciwko któr..."


In [25]:
pairs_negative_train_df.shape, pairs_negative_test_df.shape, pairs_negative_validation_df.shape

((70824, 2), (8524, 2), (6186, 2))

Sanity check zaliczony

## Zadanie 4

Train a text classifier using the Transformers library that distinguishes between the positive and the negative pairs. To make the process manageable use models of size base and a runtime providing GPU/TPU acceleration. Consult the discussions related to fine-tuning Transformer models to select sensible set of parameters. You can also run several trainings with different hyper-parameters, if you have access to large computing resources.

Dorzucam do zebranych par etykiety

In [26]:
column_count = len(pairs_positive_train_df.columns)
pairs_positive_test_df.insert(column_count, 'label', 1)
pairs_positive_train_df.insert(column_count, 'label', 1)
pairs_positive_validation_df.insert(column_count, 'label', 1)
pairs_negative_test_df.insert(column_count, 'label', 0)
pairs_negative_train_df.insert(column_count, 'label', 0)
pairs_negative_validation_df.insert(column_count, 'label', 0)

In [27]:
herbert_model_name = 'allegro/herbert-base-cased'

Robię tak jak w zalinkowanym w treści zadania [notebooku](https://github.com/apohllo/sztuczna-inteligencja/blob/master/lab5/lab_5.ipynb)

Łączymy wszystkie pary w jeden tekst z wyrażeniami: `Pytanie:` & `Odpowiedź:`, robię z tego `ds.Dataset`

In [28]:
pairs_train_df = pd.concat([pairs_positive_train_df, pairs_negative_train_df], ignore_index=True)
pairs_test_df = pd.concat([pairs_positive_test_df, pairs_negative_test_df], ignore_index=True)
pairs_validation_df = pd.concat([pairs_positive_validation_df, pairs_negative_validation_df], ignore_index=True)

In [29]:
pairs_train_df.shape

(84990, 3)

In [30]:
pairs_train_df.head(1)

,query,answer,label
0,Co jest uważane za wydatek służbowy w podróży ...,Wytyczne IRS dotyczące tematu. Ogólnie rzecz b...,1


In [31]:
def convert_to_transformers_dataset(df: pd.DataFrame) -> ds.Dataset:
    texts = []
    labels = []
    for ind, row in df.iterrows():
        query = row['query']
        answer = row['answer']
        label = row['label']
        texts.append(f'Pytanie: {query} Odpowiedź: {answer}')
        labels.append(label)
    return ds.Dataset.from_dict({
        'text': texts,
        'label': labels
    })
    

train_dataset = convert_to_transformers_dataset(pairs_train_df)
test_dataset = convert_to_transformers_dataset(pairs_test_df)
validation_dataset = convert_to_transformers_dataset(pairs_validation_df)

datasets = ds.DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": validation_dataset
})
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 84990
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10230
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 7424
    })
})

In [32]:
datasets.save_to_disk("./dumped-dataset")

Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7424/7424 [00:00<00:00, 754726.67 examples/s]


In [33]:
from transformers import AutoTokenizer

pl_tokenizer = AutoTokenizer.from_pretrained(herbert_model_name)

def tokenize_fn(sample):
    return pl_tokenizer(sample['text'], padding='max_length', truncation=True)

In [34]:
tokenized_datasets = datasets.map(tokenize_fn, batched=True)
tokenized_datasets['train']

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7424/7424 [00:01<00:00, 5780.20 examples/s]


Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 84990
})

In [35]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(herbert_model_name, num_labels=2)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [36]:
from transformers import TrainingArguments
import numpy as np

arguments = TrainingArguments(
    output_dir="./model/",
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=300,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-05,
    num_train_epochs=1,
    logging_first_step=True,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="epoch",
    fp16=False,
)

In [37]:
import evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return metric.compute(predictions=preds, references=labels)

In [39]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_datasets['train'].shuffle(seed=84),
    eval_dataset=tokenized_datasets['test'].shuffle(seed=84),
    compute_metrics=compute_metrics
)

In [40]:
# trainer.train()

^ Sypnęło brakiem RAMU, więc dalsza część rozwiązania będzie w collabie.

Jako osobny plik PDF załączam wyniki z collaba -- niestety nie udało mi się wytrenować modelu -- przyczyn mogę się tylko domyślać: być może chodzi o to niezbalansowanie zbioru. Czytałem podesłane artykuły, m.in. o `focal_loss` i dodawaniu wag do labelek, co ma pomóc modelowi poradzić sobie z tak niezbalansowanym zbiorem, ale nie widziałem w API transformers bezpośredniej możliwości zastosowania tego, a moja znajomosć TF / PyTorcha jest **skromna** ;D No nic, gdzieś w tygodniu się od kogoś dowiem jak to trzeba było zrobić. 


**Przejadę laby do końca, korzystając po prostu z niewytrenowanego modelu**


Biorę po 10 najlepszych (wg FTSa) wyników dla danego zapytania, dla każdego z tych wyników, buduję query, tokenizuję je i wrzucam do modelu i popatrzę co przewidział niewytrenowany model.

In [41]:
tokenized_validation_ds = tokenized_datasets['validation']
tokenized_validation_ds

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7424
})

In [53]:
import torch
from torch.utils.data import DataLoader

batch_size = 32
dataloader = DataLoader(tokenized_validation_ds, batch_size=batch_size, shuffle=True)

model_predictions = []
actual_labels = []
for batch in tqdm(dataloader):
    tokens = pl_tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True).to('mps')
    with torch.no_grad():
        outputs = model(**tokens)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probs, dim=-1)
    model_predictions.append(prediction)
    
    reality = batch['label']
    actual_labels.append(reality)
# model(pl_tokenizer(tokenized_validation_ds['text'][0]))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 232/232 [04:09<00:00,  1.08s/it]


In [54]:
len(model_predictions), len(actual_labels)

(232, 232)

In [61]:
model_predictions_tensor = torch.cat(model_predictions).to('cpu')
actual_labels_tensor = torch.cat(actual_labels)

In [69]:
count = model_predictions_tensor.size()[0]
hits = torch.count_nonzero(model_predictions_tensor == actual_labels_tensor).item()
hits / count

0.18036099137931033

No nie jest dobrze :D (^ accuracy na zbiorze walidacyjnym). Jednak nie ciągnę tego dalej.

## Pytania

1. Do you think simpler methods, like Bayesian bag-of-words model, would work for sentence-pair classification? Justify your answer.

Pomimo tego, że nie udało mi się wytrenować tego modelu, to zakładam, że ta metoda jest sensowna i gdyby zrobić to sensownie to można dostać wyniki nadające się do zastosowania produkcyjnego. Jezeli prównam do tego podejście BOW to moja intuicja jest taka: jako że odpowiedzi na pytania często zawierają te same tokeny co same pytania, to mogłoby to zadziałać. Czy lepiej niż Transformer -- nie sądzę, czy do produkcyjnych zastosowań? Także obstawiałbym, że nie. Natomiast chętnie posłucham omówienia tego tematu.

2. What hyper-parameters you have selected for the training? What resources (papers, tutorial) you have consulted to select these hyper-parameters?

Czytałem podrzucone artykuły, ale trenowanie modelu mi nie wyszło, więc pomijam :D 

4. Think about pros and cons of the neural-network models with respect to natural language processing. Provide at least 2 pros and 2 cons.

+: sieć ma możliwość "wykrycia głębszych" związków pomiędzy poszczególnymi tokenami w wypowiedzi, potrafi nauczyć się "głębokiej" reprezentacji danych i powiązań pomiedzy różnymi fragmentami wypowiedzi

+: patrząc na obecny rozwój modeli językowych, GPT-4, ostatnio Gemini (Google) & Grok (Musk) zdaje się, że jest to way-to-go, szczególnie gdy takie modele potrafią osiągać lepsze wyniki w testach na "wnioskowanie" i wiedzę niż 90% ludzi (odnoszę się tu do Gemini i papera który wypuściło Google Deep Mind). Na razie nie mamy innej technologi, która dawałaby podobne wyniki.

+: z tego labu: szerokość zagadnień do jakich możemy stosować te modele: można mieć "ogólnie wytrenowany" model, a następnie go "fine-tuning'ować" do konkretnego zastosowania -- "flexibility", "transfer-learning"


-: koszty (czas i pieniądze) treningu dużych modeli

-: ilość potrzebnych danych, do wytrenowania modelu nadającego się do zastosowań produkcyjnych

-: są sytuacje gdy korzystanie z NN to stanowczny overkill (mogą wystarczyć regexy :D)

-: nie do końca wiemy co się dzieje w środku tych modeli i jak one podejmują deycyzje (black boxy)